# Audio with Transformers

Dans ce notebook, nous allons utiliser la library transformers de Hugging Face pour le traitement de l'audio.

## Classification zero-shot

Commençons par un problème déjà vu en vision par ordinateur, la clasification zero-shot. Il va s'agir de donner la provenance d'un extrait sonore sans avoir eu d'entraînement sur des catégories spécifiques.

### Implémentation

Pour cela, nous utilisons le dataset [ESC-50](https://github.com/karolpiczak/ESC-50) (ashraq/esc50) qui regroupe des enregistrements de 5 secondes de 50 classes différentes.   
Pour le télécharger, utilisons la library datasets de Hugging Face : 

In [1]:
from datasets import load_dataset
from transformers import pipeline
from IPython.display import Audio as IPythonAudio

/home/aquilae/anaconda3/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset = load_dataset("ashraq/esc50",split="train[0:10]")

On peut observer les métadonnées de l'extrait audio. Le *sampling_rate* est particulièrement important. Il faut s'assurer que c'est le même que celui des données d'entraînement du modèle.

In [6]:
audio_sample = dataset[0]
audio_sample

{'filename': '1-100038-A-14.wav',
 'fold': 1,
 'target': 14,
 'category': 'chirping_birds',
 'esc10': False,
 'src_file': 100038,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.01184082, -0.10336304, -0.14141846, ...,  0.06985474,
          0.04049683,  0.00274658]),
  'sampling_rate': 44100}}

On peut écouter l'extrait audio (attention à ne pas mettre trop fort le son de votre ordinateur) avec IPython.

In [7]:
IPythonAudio(audio_sample["audio"]["array"],rate=audio_sample["audio"]["sampling_rate"])

Il est temps d'utiliser le pipeline de Hugging Face pour récuperer notre modèle.  
On utilise le modèle [CLAP](https://github.com/LAION-AI/CLAP) de LAION-AI (laion/clap-htsat-unfused).

In [9]:
audio_zero_shot = pipeline(task="zero-shot-audio-classification",model="laion/clap-htsat-unfused")

Regardons le *sampling rate* du modèle pour voir si il correspond  celui de nos données.

In [14]:
print("Sampling rate du modèle : ",audio_zero_shot.feature_extractor.sampling_rate)
print("Sampling rate de notre extrait : ",audio_sample["audio"]["sampling_rate"])

Sampling rate du modèle :  48000
Sampling rate de notre extrait :  44100


On va devoir modifier le *sampling rate* de notre dataset pour l'adapter au modèle.

In [16]:
from datasets import Audio
dataset = dataset.cast_column("audio",Audio(sampling_rate=48_000))
audio_sample = dataset[0]
print("Sampling rate de notre extrait : ",audio_sample["audio"]["sampling_rate"])

Sampling rate de notre extrait :  48000


Maintenant que les extraits et le modèle sont sur la même longueur d'onde, on va pouvoir procéder à notre classification.  
On va proposer des labels candidats (comme pour le modèle CLIP en vision).

In [18]:
candidate_labels = ["Sound of a dog","Sound of cat"]
outputs=audio_zero_shot(audio_sample["audio"]["array"],candidate_labels=candidate_labels)
print("Score de "+candidate_labels[0],outputs[0]["score"])
print("Score de "+candidate_labels[1],outputs[1]["score"])

Score de Sound of a dog 0.9805886149406433
Score de Sound of cat 0.019411340355873108


Le modèle est capable d'identifier que l'extrait audio est un aboiement de chien et pas un miaulement de chat.   
Vous pouvez tester sur vos propres extraits sonores ou sur d'autres extraits du dataset.

## Reconnaissance automatique de la parole

La detection automatique la parole est une tâche qui consiste à transcrire des paroles en texte. C'est utile pour la prise de note par la parole, pour activer des objets connectés ('Ok google', 'Hey Siri') et bien d'autres choses.

### Implémentation

Dans cet exemple, nous utilisons [LibriSpeech ASR corpus](https://www.openslr.org/12) qui contient environ 1000 heures de parole en anglais.

In [19]:
from datasets import load_dataset
dataset = load_dataset("librispeech_asr",split="train.clean.100",streaming=True,trust_remote_code=True)

In [25]:
example = next(iter(dataset))
example

{'file': '374-180298-0000.flac',
 'audio': {'path': '374-180298-0000.flac',
  'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
         -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'CHAPTER SIXTEEN I MIGHT HAVE TOLD YOU OF THE BEGINNING OF THIS LIAISON IN A FEW LINES BUT I WANTED YOU TO SEE EVERY STEP BY WHICH WE CAME I TO AGREE TO WHATEVER MARGUERITE WISHED',
 'speaker_id': 374,
 'chapter_id': 180298,
 'id': '374-180298-0000'}

In [24]:
from IPython.display import Audio as IPythonAudio

IPythonAudio(example["audio"]["array"],rate=example["audio"]["sampling_rate"])

Nous allons utiliser le modèle [Whisper](https://openai.com/index/whisper/) (distil-whisper/distil-small.en) de OpenAI qui est conçu pour la reconnaissance de paroles anglaises. C'est une version réduite du modèle original.   
Construisons maintenant notre pipeline Hugging Face.

In [27]:
reco_parole = pipeline(task="automatic-speech-recognition",model="distil-whisper/distil-small.en")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
print("Sampling rate du modèle : ",reco_parole.feature_extractor.sampling_rate)
print("Sampling rate de notre extrait : ",example['audio']['sampling_rate'])

Sampling rate du modèle :  16000
Sampling rate de notre extrait :  16000


Les *sampling rate* sont identiques, pas besoin de modifier quoi que ce soit.

In [37]:
output=reco_parole(example["audio"]["array"])
print("Texte transcrit : ",output['text'])
print("Texte de base : ",example["text"].lower())

Texte transcrit :   Chapter 16 I might have told you of the beginning of this liaison in a few lines, but I wanted you to see every step by which we came. I too agree to whatever Marguerite wished.
Texte de base :  chapter sixteen i might have told you of the beginning of this liaison in a few lines but i wanted you to see every step by which we came i to agree to whatever marguerite wished


Comme vous pouvez le voir, la transcription est assez fidèle à l'original. 

## Transformation de texte en dialogue

Cette tâche est l'inverse de la tâche précédente. Ici, on va donner un texte en entrée et le modèle va nous ressortir un audio d'une personne disant ce texte. 

### Implémentation

Nous utilisons le modèle [vits](https://github.com/jaywalnut310/vits) de kakao-enterprise (kakao-enterprise/vits-ljs). 

In [5]:
text_parole = pipeline("text-to-speech", model="kakao-enterprise/vits-ljs")

Some weights of the model checkpoint at kakao-enterprise/vits-ljs were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.

Essayons de génerer une phrase en français.

In [6]:
text = """Ce cours de deep learning est incroyable."""
generated_parole = text_parole(text)
IPythonAudio(generated_parole["audio"][0],rate=generated_parole["sampling_rate"])

Comme vous pouvez le constater, ce n'est pas terrible car le modèle est entraîné sur des données en anglais. Si vous voulez génerer du français, il faudra trouver un modèle adapté pour cela.  
Essayons maintenant sur une phrase en anglais : 

In [7]:
text = """This deep learning course is fantastic."""
generated_parole = text_parole(text)
IPythonAudio(generated_parole["audio"][0],rate=generated_parole["sampling_rate"])

C'est beaucoup mieux !   

**Note 1** : Vous pouvez aussi combiner plusieurs modèles, par exemple vous prenez votre phrase en français, vous la traduisez en anglais puis vous générer l'audio correspondant.

**Note 2** : Si vous souhaitez générer du son (musique, son d'ambiance, bruit etc ...) qui ne soit pas de la parole, vous devez plutôt regarder la catégorie Text-to-Audio sur Hugging Face.